In [1]:
import scipy.linalg as sc_linalg


def check_riccati(P, A, B, Q, R, S):
    return np.linalg.norm(Q + A.T @ P @ A - (S + A.T @ P @ B) @ sc_linalg.inv(R + B.T @ P @ B) @ (S + B.T @ P @ A) - P)


def check_riccati_with_sol_without_S(P, A, B, Q, R):
    a_equa = B[0, 0] ** 2
    b_equa = R[0, 0] * (1 - A[0, 0] ** 2) -  B[0, 0] ** 2 * Q[0, 0]
    c_equa = - R[0, 0] * Q[0, 0]

    delta_equa = b_equa ** 2 - 4 * a_equa * c_equa

    return np.linalg.norm (- b_equa / (2 * a_equa) - np.sqrt(delta_equa) / (2 * a_equa) - P[0, 0])

In [4]:
import numpy as np

A = np.ones((1, 1)) * 1 # 5 * (2 * np.random.random(size=(1, 1)) - 1)
B = np.ones((1, 1)) * 1 # 5 * (2 * np.random.random(size=(1, 1)) - 1)
Q = np.ones((1, 1)) * - 1 # 5 * (2 * np.random.random(size=(1, 1)) - 1)
R = np.ones((1, 1)) * - 1 # 5 * (2 * np.random.random(size=(1, 1)) - 1)
S = np.zeros((1, 1))

print("Transition: s' = As + Ba")
print(f"Transition: s' = {np.around(A[0, 0], 2)}s + {np.around(B[0, 0], 2)}a")
print("Reward: Qs² + Ra² + 2 Sas")
print(f"Reward: {np.around(Q[0, 0], 2)}s² + {np.around(R[0, 0], 2)}a² + 2 {np.around(S[0, 0], 2)}as")
print()

P = sc_linalg.solve_discrete_are(A, B, Q, R, s=S)
riccati_norm_error = check_riccati(P, A, B, Q, R, S)

assert riccati_norm_error < 1e-9, f"Riccati equation is not verified: {riccati_norm_error}"

S_hat = S + A.T @ P @ B
R_hat = R + B.T @ P @ B
K = sc_linalg.inv(R_hat) @ S_hat

Transition: s' = As + Ba
Transition: s' = 1.0s + 1.0a
Reward: Qs² + Ra² + 2 Sas
Reward: -1.0s² + -1.0a² + 2 0.0as



In [9]:
a_equa = B[0, 0] ** 2
b_equa = R[0, 0] * (1 - A[0, 0] ** 2) -  B[0, 0] ** 2 * Q[0, 0]
c_equa = - R[0, 0] * Q[0, 0]

delta_equa = b_equa ** 2 - 4 * a_equa * c_equa

- b_equa / (2 * a_equa) + np.sqrt(delta_equa) / (2 * a_equa)

0.6180339887498949

In [5]:
R_hat, K, P

(array([[-2.61803399]]), array([[0.61803399]]), array([[-1.61803399]]))

In [12]:
state = np.array([4])

for i in range(10):
    action = - K @ state
    action = action + 1 if action > 0 else action - 1
    next_state = A @ state + B @ action
    reward = state.T @ Q @ state + 2 * state.T @ S @ action + action.T @ R @ action

    print(
        "State:", np.around(state[0], 2), 
        "action:", np.around(action[0], 2), 
        "next state:", np.around(next_state[0], 2), 
        "reward:", np.around(reward, 2)
    )
    state = next_state

State: 4 action: -3.47 next state: 0.53 reward: -28.06
State: 0.53 action: -1.33 next state: -0.8 reward: -2.04
State: -0.8 action: 1.49 next state: 0.7 reward: -2.87
State: 0.7 action: -1.43 next state: -0.73 reward: -2.53
State: -0.73 action: 1.45 next state: 0.72 reward: -2.65
State: 0.72 action: -1.44 next state: -0.73 reward: -2.6
State: -0.73 action: 1.45 next state: 0.72 reward: -2.62
State: 0.72 action: -1.45 next state: -0.72 reward: -2.62
State: -0.72 action: 1.45 next state: 0.72 reward: -2.62
State: 0.72 action: -1.45 next state: -0.72 reward: -2.62


In [33]:
A * R, R + B**2 @ P

(array([[-14.51282393]]), array([[-86.65466183]]))

In [21]:
import control

K_control, P_control, _ = control.dlqr(A, B, Q, R, S)

print(P_control[0, 0], P[0, 0])
print(K_control[0, 0], K[0, 0])

-3.408517062164548 -3.408517062164548
1.0286411805850395 1.0286411805850393


In [36]:
_

array([-0.08784434+0.j])

In [1]:
import numpy as np 

from pbo.environment.linear_quadratic import LinearQuadraticEnv
import control
import scipy

mdp = LinearQuadraticEnv(
    A=2 * np.random.random(size=(1, 1)) - 1,
    B=2 * np.random.random(size=(1, 1)) - 1,
    Q=2 * np.random.random(size=(1, 1)) - 1,
    R=2 * np.random.random(size=(1, 1)) - 1,
    S=np.zeros((1, 1)),
    max_pos=10,
    max_action=10,
    initial_state=np.array([3]),
)

# dlqr minimises J = \\Sum_0^\\infty (s[n]' Q_given s[n] + a[n]' R_given a[n] + + 2 s[n]' S_given a[n])
# here we want to maximises the quantity 
# this is why we need to have Q_given = -Q, R_given = -R, S_given = -S
K, S, E = control.dlqr(mdp.A, mdp.B, -mdp.Q, -mdp.R, -mdp.S)
# scipy.linalg.solve_discrete_are(mdp.A, B, Q, R, e=E, s=S)

print("Transition: s' = As + Ba")
print(f"Transition: s' = {np.around(mdp.A[0, 0], 2)}s + {np.around(mdp.A[0, 0], 2)}a")
print("Reward: Qs² + Ra² + 2 Sas")
print(f"Reward: {np.around(mdp.Q[0, 0], 2)}s² + {np.around(mdp.R[0, 0], 2)}a² + 2 {np.around(mdp.S[0, 0], 2)}as")

Transition: s' = As + Ba
Transition: s' = -0.49s + -0.49a
Reward: Qs² + Ra² + 2 Sas
Reward: -0.13s² + -0.62a² + 2 0.0as


In [22]:
def check_riccati(P, A, B, Q, R, S):
    return np.linalg.norm(Q + A.T @ P @ A - (S + A.T @ P @ B) @ scipy.linalg.inv(R + B.T @ P @ B) @ (S + B.T @ P @ A) - P)

In [23]:
check_riccati(P, mdp.A, mdp.B, mdp.Q, mdp.R, mdp.S)

2.7755575615628914e-17

In [19]:
S_hat = mdp.S + mdp.B * S * mdp.A
R_hat = mdp.R + mdp.B * S * mdp.B
K_hat = S_hat[0, 0] / R_hat[0, 0]
print("K from theory:", K_hat)
print("K given:", K[0, 0])

K from theory: 2.4123148502450835
K given: 1.377419857182332


In [62]:
import control

K, S, E = control.dlqr(mdp.A, mdp.B, mdp.Q, mdp.R)
# K = np.array([[0]])

state = mdp.reset()
terminal = False
step = 0

while not terminal and step < 10:
    next_state, reward, terminal, _ = mdp.step(- K @ state)

    print(f"State: {np.around(state[0], 2)}", end=" ")
    print(f"Action: {np.around((- K @ state)[0], 2)}", end=" ")
    print(f"Reward: {np.around(reward, 4)}")

    state = next_state
    step += 1

State: 3 Action: 0.01 Reward: -0.9793
State: 0.06 Action: 0.0 Reward: -0.0004
State: 0.0 Action: 0.0 Reward: -0.0
State: 0.0 Action: 0.0 Reward: -0.0
State: 0.0 Action: 0.0 Reward: -0.0
State: 0.0 Action: 0.0 Reward: -0.0
State: 0.0 Action: 0.0 Reward: -0.0
State: 0.0 Action: 0.0 Reward: -0.0
State: 0.0 Action: 0.0 Reward: -0.0
State: 0.0 Action: 0.0 Reward: -0.0


In [67]:
S

array([[0.10885098]])

In [64]:
K[0, 0]

-0.0021193163077151203

In [65]:
mdp.A * 3 + mdp.B * 1.91

array([[-1.41677715]])

In [66]:
(mdp.A[0, 0] * S[0, 0] * mdp.B[0, 0]) / (mdp.R[0, 0] + mdp.B[0, 0] ** 2)

-0.0012833809110176309